In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('processRenorms')

In [6]:
enss=['b','c','d']
ens2RCs={ens:{} for ens in enss}

path='data_aux/RCs/perturb/paperRenorms.txt'
with open(path,'r') as f:
    rcs=f.read().splitlines()

Zs=['Zqq(mu=nu)','Zqq(mu!=nu)','Zqq^s(mu=nu)','Zqq^s(mu!=nu)','Zqg(mu!=nu)','Zgq(mu=nu)','Zgq(mu!=nu)']
for iZ,Z in enumerate(Zs):
    for iens,ens in enumerate(enss):
        t=rcs[iZ*4+iens]
        if '(' not in t:
            ens2RCs[ens][Z] = float(t)
            ens2RCs[ens][Z+'_err'] = 0
        else:
            ind_dot=t.find('.')
            ind_b=t.find('(')
            ind_k=t.find(')')
            ens2RCs[ens][Z] = float(t[:ind_b])
            ens2RCs[ens][Z+'_err'] = float(t[ind_b+1:ind_k]) * 10**(-(ind_b-ind_dot)+1)

for ens,ensL in zip(enss,['cB64','cC80','cD96']):
    path=f'data_aux/RCs/perturb/Zgg_c{ens.upper()}4.txt'
    with open(path,'r') as f:
        t=f.read().splitlines()
        for i,row in enumerate(t):
            # if i==0:
            #     continue
            nstout,mean,err=[float(ele) for ele in row.split('\t')]
            nstout=int(nstout)
            assert(nstout==i)
            ens2RCs[ens][f'Zgg^{i}(mu!=nu)']=mean
            ens2RCs[ens][f'Zgg^{i}(mu!=nu)_err']=err  
            
for ens in enss:
    path=f'data_aux/RCs/perturb/Zgg_mu_eq_nu_c{ens.upper()}4.txt'
    with open(path,'r') as f:
        t=f.read().splitlines()
        for i,row in enumerate(t):
            # if i==0:
            #     continue
            nstout,mean,err=[float(ele) for ele in row.split('\t')]
            nstout=int(nstout)
            assert(nstout==i)
            ens2RCs[ens][f'Zgg^{i}(mu=nu)']=mean
            ens2RCs[ens][f'Zgg^{i}(mu=nu)_err']=err
            
for ens in enss:
    path=f'data_aux/RCs/perturb/Zqq_Zqg_Zgq_mu_eq_nu_c{ens.upper()}4.txt'
    with open(path,'r') as f:
        t=f.read().splitlines()
        for i,row in enumerate(t):
            key,val=row.replace(" ", "").split('=')
            if '(' not in val:
                mean=float(val); err=0
            else:
                ind_dot=val.find('.')
                ind_b=val.find('(')
                ind_k=val.find(')')
                mean = float(val[:ind_b])
                err = float(val[ind_b+1:ind_k]) * 10**(-(ind_b-ind_dot)+1)
            ens2RCs[ens][f'{key}(mu=nu)']=mean
            ens2RCs[ens][f'{key}(mu=nu)_err']=err
            
path='data_aux/RCs.pkl'
with open(path,'wb') as f:
    pickle.dump(ens2RCs,f)

In [14]:
ens2RCs={}
path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs=pickle.load(f)

Zs=['Zqq(mu=nu)','Zqq(mu!=nu)','Zqq^s(mu=nu)','Zqq^s(mu!=nu)','Zqg(mu=nu)','Zqg(mu!=nu)','Zgq(mu=nu)','Zgq(mu!=nu)']
precisions=[1,1,2,1,0,0,3,3]
txts=[]
for iZ,Z in enumerate(Zs):
    t=', '.join([yu.un2str(ens2RCs[ens][Z],ens2RCs[ens][Z+'_err'],precisions[iZ]) if ens2RCs[ens][Z+'_err']!=0 else str(ens2RCs[ens][Z]) for ens in enss])
    txts.append(f'{Z}: {t}')
txts.append('')
for stout in [5,7,9,10,11,13,15,17,20]:
    Z=f'Zgg^{stout}(mu!=nu)'
    t=', '.join([yu.un2str(ens2RCs[ens][Z],ens2RCs[ens][Z+'_err'],1 if ens=='b' else 2) for ens in enss])
    txts.append(f'{Z}: {t}')
    Z=f'Zgg^{stout}(mu=nu)'
    t=', '.join([yu.un2str(ens2RCs[ens][Z],ens2RCs[ens][Z+'_err'],1 if ens=='b' else 2) for ens in enss])
    txts.append(f'{Z}: {t}')
    
yu.save_txt('/p/project1/ngff/li47/code/glwc3/project2/05_moments/data_aux/RCs.txt',txts)

In [2]:
# preliminary non-pertubative result
enss=['b','c','d']
ens2name={'b':'B12','c':'C24','d':'D24'}
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    path=f'data_aux/RCs/nonper/ZMSbar_matrix_final_results_nd_c{ens2name[ens]}.txt'
    with open(path,'r') as f:
        t=f.read().splitlines()
        for i,row in enumerate(t[1:]):
            stout,Zqqs,Zqqs_err,Zqg,Zqg_err,Zgq,Zgq_err,Zgg,Zgg_err=[float(ele) for ele in row.split('\t')]
            stout=int(stout)
            
            munu='(mu!=nu)'
            ens2RCs[ens][f'Zqq^s^{stout}{munu}']=Zqqs
            ens2RCs[ens][f'Zqq^s^{stout}{munu}_err']=Zqqs_err
            ens2RCs[ens][f'Zqg^{stout}{munu}']=Zqg
            ens2RCs[ens][f'Zqg^{stout}{munu}_err']=Zqg_err
            ens2RCs[ens][f'Zgq^{stout}{munu}']=Zgq
            ens2RCs[ens][f'Zgq^{stout}{munu}_err']=Zgq_err
            ens2RCs[ens][f'Zgg^{stout}{munu}']=Zgg
            ens2RCs[ens][f'Zgg^{stout}{munu}_err']=Zgg_err
ens2RCs_pre=ens2RCs

path='data_aux/RCs_pre.pkl'
with open(path,'wb') as f:
    pickle.dump(ens2RCs_pre,f)

ens2RCs={}
path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs=pickle.load(f)
    
stouts=np.arange(0,40+1,2)
def getme(case):
    mean=[ens2RCs_pre[ens][f'{case}^{stout}{munu}'] for stout in stouts]
    err=[ens2RCs_pre[ens][f'{case}^{stout}{munu}_err'] for stout in stouts]
    m=[ens2RCs[ens][f'{case}{munu}'] if 'Zgg' not in case else ens2RCs[ens][f'{case}^{stout}{munu}'] for stout in stouts]
    e=[ens2RCs[ens][f'{case}{munu}_err'] if 'Zgg' not in case else ens2RCs[ens][f'{case}^{stout}{munu}_err'] for stout in stouts]
    return mean,err,m,e

fig,axs=yu.getFigAxs(4,len(enss),sharex=True,sharey='row',Lrow=4,Lcol=6)
yu.addColHeader(axs,[yu.ens2label[ens] for ens in enss])
ylabels=['Zqq^s','Zqg','Zgq','Zgg']
axs[0,0].set_ylim([1.1,1.25])
axs[1,0].set_ylim([-0.1,0.1])
axs[2,0].set_ylim([-0.1,0.3])
axs[3,0].set_ylim([0.6,1])
for iens,ens in enumerate(enss):
    axs[3,iens].set_xlabel(r'$n_{\rm stout}$')
    for i,ylabel in enumerate(ylabels):
        ax=axs[i,iens]
        yu.addRefLine(ax,10,'v')
        if iens==0:
            ax.set_ylabel(ylabel)
        mean,err,m,e=getme(ylabel)
        plt_x=stouts; plt_y=m; plt_yerr=e
        ax.errorbar(plt_x,plt_y,plt_yerr,color='b',mfc='white')
        plt_x=stouts; plt_y=mean; plt_yerr=err
        ax.errorbar(plt_x,plt_y,plt_yerr,color='r')
fig.suptitle(r'$\mu\neq\nu$')
yu.finalizePlot('RCs_per_vs_nonp')